In [1]:
# !unzip robohearts.zip
#%cd robohearts/

In [2]:
import gym
import multiprocessing
from gymhearts.Hearts import *
from gymhearts.Agent.human import Human
from gymhearts.Agent.random_agent import RandomAgent
from gymhearts.Agent.monte_carlo_nn import MonteCarloNN
from gymhearts.Agent.hand_approx import save_model
from tqdm import tqdm_notebook

In [3]:
NUM_TESTS = 10
NUM_EPISODES = 1000
TRAINING_ITERS = 10000
MAX_SCORE = 100

run_train = True

playersNameList = ['MonteCarlo', 'Rando', 'Randy', 'Randall']
agent_list = [0, 0, 0, 0]

# Human vs Random
# in_hand, in_play, played_cards, won_cards, scores
mc_config = {
    'print_info' : False,
    'epsilon' : .05,
    'gamma' : .9,
    'alpha': 1e-4,
    'feature_list' : ['in_hand', 'in_play']
}
agent_list[0] = MonteCarloNN(playersNameList[0], mc_config)
agent_list[1] = RandomAgent(playersNameList[1], {'print_info' : False})
agent_list[2] = RandomAgent(playersNameList[2], {'print_info' : False})
agent_list[3] = RandomAgent(playersNameList[3], {'print_info' : False})

In [4]:
# TRAIN THE MONTE CARLO AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
if run_train:
    for trn_episode in tqdm_notebook(range(TRAINING_ITERS)):
        observation = env.reset()
        history = []
        while True:
            #env.render()

            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            # update my agent before clearing state
            if now_event == 'RoundEnd':
                agent_list[0].update_weights(history, -reward['MonteCarlo'])
                history = []
            if now_event == 'GameOver':
                  break
            if observation['event_name']=='PlayTrick' and observation['data']['playerName'] == 'MonteCarlo':
                # don't add score, they don't change till at end of round
                history.append((observation, agent_list[0].played_cards, agent_list[0].won_cards))

            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)

            observation, reward, done, info = env.step(action)
    save_model(agent_list[0].nn)

TypeError: array() missing required argument 'object' (pos 1)

In [ ]:
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'MonteCarlo')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [ ]:
# Uncomment this line for pretrained weights
# weights = [-0.46031637, -1.02296217, -1.64597146, 0.50871499, 0.05907032, -0.04527117,
# -2.67590788, -1.78400492, -0.08667306, 0.48108891, 0.66066313, -1.57675411,
# -0.56494518, -0.07736412, -0.3257198, -0.65003209, -0.63740714, 0.44494984,
# -0.1545964, 0.67457139, 2.31472314, 0.8694452, -2.29173301, 0.52783125,
# -0.86950875, -1.77655688, -3.29970913, -0.242993, -1.57548922, -1.34238258,
# 0.36816378, -3.23065985, -0.07919411, -2.1089143, -3.12815169, -0.74580836,
# 0.98398675, -0.75271283, -0.81051661, -0.60567687, -3.42010519, -0.63186969,
# -2.02352157, -0.27534069, -0.28736574, -1.15836776, -3.28679005, -0.33767846,
# -0.41568405, 0.2782292, -1.23761129, -1.80559854]

# EVALUATE THE MONTE CARLO AGENT
    
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = MonteCarloNN(playersNameList[0], params={'nn_path': 'mlp', 'in_hand': True, 'in_play': True,})
mc_wins = [0] * NUM_TESTS        
pool = multiprocessing.Pool(processes=NUM_TESTS)
mc_wins = pool.map(run_test, mc_wins)
print(mc_wins)
pool.close()
pool.join()

In [ ]:
def run_test_2(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'Randman')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [ ]:
# EVALUATE THE RANDOM AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
playersNameList[0] = 'Randman'
agent_list[0] = RandomAgent(playersNameList[0])
rand_wins = [0] * NUM_TESTS
        
pool = multiprocessing.Pool(processes=NUM_TESTS)
rand_wins = pool.map(run_test_2, rand_wins)
print(rand_wins)
pool.close()
pool.join()

In [ ]:
print(f"Monte Carlo won {sum(mc_wins)/len(mc_wins)} times on average :: {str(mc_wins)}")
print(f"Random won {sum(rand_wins)/len(rand_wins)} times on average :: {str(rand_wins)}")
# print(f"\n\nThe Monte Carlo weights are: {str(weights)}")